In [1]:
# python
import sys
import os
import importlib
# columnar analysis
import awkward as ak
from coffea import processor, nanoevents
# local
sidm_path = str(sys.path[0]).split("/sidm")[0]
if sidm_path not in sys.path: sys.path.insert(1, sidm_path)
from sidm.tools import llpnanoaodschema, sidm_processor
# always reload local modules to pick up changes during development
importlib.reload(llpnanoaodschema)
importlib.reload(sidm_processor)

<module 'sidm.tools.sidm_processor' from '/home/cms-jovyan/SIDM/sidm/tools/sidm_processor.py'>

In [2]:
files = [
    "root://xcache//store/group/lpcmetx/SIDM/Backgrounds/2018_v2/Skims/QCD_Pt-15To20/skimmed_output_446.root",
    "root://xcache//store/group/lpcmetx/SIDM/Backgrounds/2018_v2/Skims/QCD_Pt-15To20/skimmed_output_447.root"
]
fileset = {
    "QCD_Pt15To20": {
        'files': files[:1],
        #'files': files[1:],
        "metadata": {
            "skim_factor": 1.0,
            "is_data": False,
            "year": "2018",
        },
    },
}
fileset

{'QCD_Pt15To20': {'files': ['root://xcache//store/group/lpcmetx/SIDM/Backgrounds/2018_v2/Skims/QCD_Pt-15To20/skimmed_output_446.root'],
  'metadata': {'skim_factor': 1.0, 'is_data': False, 'year': '2018'}}}

In [3]:
p = sidm_processor.SidmProcessor(
    ["baseNoLj"],
    ["muon_base"],
    verbose=True,
)

In [4]:
runner = processor.Runner(
    executor=processor.IterativeExecutor(),
    schema=llpnanoaodschema.LLPNanoAODSchema,
    maxchunks=1,
    skipbadfiles=True
)

In [5]:
output = runner.run(fileset, treename='Events', processor_instance=p)

Output()

Output()

/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing 
cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(

/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing 
cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(

/usr/local/lib/python3.12/site-packages/awkward/_nplikes/array_module.py:285: RuntimeWarning: invalid value 
encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))

...

Applying dsaMuons displaced ID

Applying dsaMuons all

Applying cut: pass triggers

Applying cut: PV filter

QCD_Pt15To20 is simulation. Scaling histograms or cutflows according to lumi*xs.


In [6]:
events_446 = nanoevents.NanoEventsFactory.from_root(
    {files[0]: "Events"},
    schemaclass=llpnanoaodschema.LLPNanoAODSchema,
).events()

events_447 = nanoevents.NanoEventsFactory.from_root(
    {files[1]: "Events"},
    schemaclass=llpnanoaodschema.LLPNanoAODSchema,
).events()

/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


In [7]:
run = 446
for events in [events_446, events_447]:
    print(run)
    mu = events.Muon
    matched_dsa = mu.matched_dsa_muons
    #print("Events:", len(events))
    #print("num muons:", ak.num(matched_dsa, axis=1))
    #print("num matched DSA:", ak.num(matched_dsa, axis=2))
    #print("num matched DSA w/o none:", ak.num(ak.drop_none(matched_dsa), axis=2))
    ##print("Matched DSA pt > 0", matched_dsa.pt > 0)
    #print(ak.is_none(matched_dsa, axis=2).show())
    dsa = events.DSAMuon
    matched_pf = dsa.matched_muons
    print(matched_pf.show())
    print(ak.is_none(matched_pf, axis=2).show())
    print(ak.is_none(matched_pf, axis=2).show())
    print()
    run += 1

#ak.all(
#        (objs["dsaMuons"].good_matched_muons.numMatch < 1) | (ak.is_none(objs["dsaMuons"].good_matched_muons, axis=2)),
#        axis=2

446
[[[LLPMuon, ..., None], ...],
 [[LLPMuon, ...]]]
None
[[[False, False, True, True, True], [False, False, True, True, True]],
 [[False, True, True, True, True]]]
None
[[[False, False, True, True, True], [False, False, True, True, True]],
 [[False, True, True, True, True]]]
None

447
[[[LLPMuon, ..., None], ...],
 [[LLPMuon, ..., None], ...]]
None
[[[False, True, True, True, True], [...], [False, True, True, True, True]],
 [[False, False, True, True, True], [False, False, True, True, True]]]
None
[[[False, True, True, True, True], [...], [False, True, True, True, True]],
 [[False, False, True, True, True], [False, False, True, True, True]]]
None



In [8]:
matched_dsa = events_447.Muon.matched_dsa_muons
print("Matched DSA:", ak.num(matched_dsa, axis=2))
print("Matched DSA pt:", matched_dsa.pt)

Matched DSA: [[5], [5, 5]]
Matched DSA pt: [??, ??]
